# P5: Photometric Stereo and Structure from Motion

## P5.1 Lighting a Scene

In class we talked about the properties of light. One of the topics we discussed was how we could compute the scene lighting of a Lambertian object from its normals and its albedo (and assuming we know where the light is). This process is relatively "simple" once you understand what's going on. For a Lambertian object, the amount of light collected by a light source is proportional to the cosine of the angle between the surface and the light source. So, for every point on the surface $\mathbf{x}$:

$$ \text{Observed Brightness}(\mathbf{x}) = \text{Albedo}(\mathbf{x})\cos(\theta_\mathbf{x}) = \text{Albedo}(\mathbf{x}) \left[ \hat{l} \cdot \hat{n}(\mathbf{x}) \right] $$

where $\hat{l}$ is the *light direction* (normalized) and $\hat{n}(\mathbf{x})$ is the *surface normal* at location $\mathbf{x}$.

For this breakout session, I have provided you with an *albedo matrix* `albedos` (of dimension $m \times n$) and the accompanying *normals* `normals` (of dimension $m \times n \times 3$, where the normal vector at coordinate `i, j` is `normals[i, j]`).

**TASK** Write a function `light_scene` that computes an $m \times n$ "lit scene" image from (1) the albedos, (2) the normals, and (3) the light directions.

I have provided you with both data and some partially-implemented functions for this process. If your `light_scene` function is implemented correctly, the `light_scene_interactive` function I have included below should instantiate an interactive widget with which you can move the light around with sliders and see the effect.

**PLOTS** Include 4 different lightings of the "bunny" scene in your writeup.

*Data and Images* from: 
- Buddah: https://courses.cs.washington.edu/courses/csep576/05wi/projects/project3/project3.htm
- Scholar: http://vision.seas.harvard.edu/qsfs/Data.html

In [ ]:
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load Data
stanford_bunny_dat = np.load('stanford_bunny_dat.npz')
normals = stanford_bunny_dat['arr_0'][:, :, :3]
albedos = stanford_bunny_dat['arr_0'][:, :, 3]

# Function you will be implementing
def light_scene(albedos, normals, light_direction):
    raise NotImplementedError()

In [ ]:
# Visualization Functions
def visualize_normals(normals, ax=None):
    """Visualize a 'normals' matrix using common 'look'."""
    if ax is None:
        plt.figure(dpi=300)
    plt.imshow(0.5 * normals + 0.5)
    
def light_scene_interactive(albedos, normals):
    # Create a function to generate relit scene and plot
    def inter(th_x, th_y):
        sth_x = np.sin(th_x)
        sth_y = np.sin(th_y)
        light_direction = [
            sth_x, sth_y, np.sqrt(1 - sth_x**2 - sth_y**2)
        ]
        ld = light_direction
        print(f"Light Direction: x={ld[0]:0.3f}, y={ld[1]:0.3f}, z={ld[2]:0.3f}")

        plt.figure(2)
        imobj = plt.imshow(light_scene(albedos, normals, light_direction),
               vmin=0.0, vmax=0.7, cmap='gray')
        plt.show()

    return interactive(inter, th_x=(-0.8, 0.8, 0.05), th_y=(-0.8, 0.8, 0.05))

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(albedos)
ax = plt.subplot(1, 2, 2)
visualize_normals(normals, ax=ax)

light_scene_interactive(albedos, normals)

## P5.2 Photometric Stereo

Now that you see how to light a scene, we will now investigate how to invert this process: a technique known as *photometric stereo*.

### P5.2.1 Using provided data

I have provided you with some data in the folders `buddah` and `scholar` which contain grayscale `.tga` images and a file `light_directions.txt`. Each line of `light_directions.txt` is a 3-element vector indicating the direction of the light source from the object corresponding to that image. **IMPORTANT:** the light directions for the buddah data are in numerical order of the images. `light_directions[0]` corresponds to `buddah.0.tga` and so on. Sorting by alphabetical order on some file systems will place `buddah.10.tga` before `buddah.2.tga`: **this is incorrect and will result in incorrect results**.

**TASK** Implement photometric stereo as discussed in `L18`.

**NOTE**: This function may return incorrect values where the albedo is zero. Feel free to add small values to the albedo, or add "fake" `[0, 0, 0]` values for the normals where they would otherwise be not well defined.

**PLOTS** Plot (1) the computed normals, (2) the computed albedo, and (3) the reconstructed depth image (via the `integrateFrankot` function provided below) for each of the `buddah` and `scholar` datasets.

I have included the `integrateFrankot` function below and an example using the "bunny" data from above to reconstruct the depth image corresponding to your computed surface normals.  

In [ ]:
## You should be using grayscale images for this assignment

import numpy as np
from PIL import Image

def load_image_gray(filepath):
    """Loads an image into a numpy array.
    Note: image will have 3 color channels [r, g, b]."""
    img = Image.open(filepath)
    img = np.asarray(img).astype(np.float)/255
    if len(img.shape) > 2:
        return img[:, :, 0]
    else:
        return img

In [ ]:
# The integrateFrankot function is from the CMU Computer Vision course

# ##################################################################### #
# 16385: Computer Vision Homework 4
# Carnegie Mellon University
# Spring 2020
# ##################################################################### #

import numpy as np
from skimage.io import imread
from skimage.color import rgb2xyz
from matplotlib import pyplot as plt
from scipy.sparse import kron as spkron
from scipy.sparse import eye as speye
from scipy.sparse.linalg import lsqr as splsqr
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import warnings

def integrateFrankot(normals):

    """
    Implement the Frankot-Chellappa algorithm for enforcing integrability
    and normal integration

    Parameters
    ----------
    normals: matrix of normal vectors
    
    Returns
    ----------
    z: numpy.ndarray
        The image, of the same size as the derivatives, of estimated depths
        at each point

    """
    zx = -normals[:, :, 0]
    zy = normals[:, :, 1]
    pad = max(normals.shape)

    # Raise error if the shapes of the gradients don't match
    if not zx.shape == zy.shape:
        raise ValueError('Sizes of both gradients must match!')

    # Pad the array FFT with a size we specify
    h, w = pad, pad

    # Fourier transform of gradients for projection
    Zx = np.fft.fftshift(np.fft.fft2(zx, (h, w)))
    Zy = np.fft.fftshift(np.fft.fft2(zy, (h, w)))
    j = 1j

    # Frequency grid
    [wx, wy] = np.meshgrid(np.linspace(-np.pi, np.pi, w),
                           np.linspace(-np.pi, np.pi, h))
    absFreq = wx**2 + wy**2

    # Perform the actual projection
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        z = (-j*wx*Zx-j*wy*Zy)/absFreq

    # Set (undefined) mean value of the surface depth to 0
    z[0, 0] = 0.
    z = np.fft.ifftshift(z)

    # Invert the Fourier transform for the depth
    z = np.real(np.fft.ifft2(z))
    z = z[:zx.shape[0], :zx.shape[1]]

    return z

# An example showing how this works with the "bunny dataset"
stanford_bunny_dat = np.load('stanford_bunny_dat.npz')
normals = stanford_bunny_dat['arr_0'][:, :, :3]
albedos = stanford_bunny_dat['arr_0'][:, :, 3]
depth = integrateFrankot(normals)
plt.figure(figsize=(8, 4))
ax = plt.subplot(1, 2, 1)
visualize_normals(normals, ax=ax)
plt.subplot(1, 2, 2)
plt.imshow(depth)
None

### P5.2.2 Imperfections in Photometric Stereo

Using photometric stereo, you have just computed the surface normals and albedo of a few objects. However, for photometric stereo to work, we make some significant assumptions about the structure of the scene and the material properties of the object. In this question, we will use the computed normals and albedos to relight the scene and you will comment on the differences between the "reconstructed" image as compared to the original.

**TASK** Using your answer to P5.1, relight the scholar object to approximately recreate `scholar/image_11.png` (using `light_positions[10]` from the `scholar` data).

**PLOTS** Plot (1) the original `scholar/image_11.png`, (2) your reconstructed "relit" image corresponding to that scene, and (3) the difference between the two images (use `cmap='PiYG', vmin=-0.3, vmax=0.3` as arguments to `imshow`).

In ideal photometric stereo, these images should be identical. However, you should notice some differences between your image and the reconstruction, particularly to the lower-left of the face.

**QUESTION** Explain (in 2–3 sentences) why there is a significant difference between the two images to the lower-left of the scholar's face.

**QUESTION** Provide an explanation (in 2–3 sentences) why the body of the scholar is (on average) darker in the original image than in the reconstruction. (There are a few potential explanations to this question.)

### P5.2.3 Lambertian Objects in Blender

I have provided you with a scene in Blender `photometric_stereo.blend` that you will use to generate data of your own. The scene is already arranged so that you can generate data: move around "Light Source", render the image, and record the "light direction" in `light_positions.txt`. The scene is structured such that simply including the x, y, z coordinates of the light in that file will match the format of the other datasets you have been provided. **Important**: for photometric stereo, the light positions need to be normalized! If you are not normalizing your light directions before computing the albedo and normals, you will need to do so. Also, it is recommended that you move the light only in X and Y, so that you have one fewer variable to worry about.

**TASK & PLOTS** Generate at least 4 images using Blender for different light positions and include plots of those images.

**PLOTS** Use photometric stereo to compute and plot the (1) albedos, (2) normals, and (3) depth for the blender scene.

### P5.2.4 Non-Lambertian Objects in Blender

Modify the material of the central object in blender (the "Icosphere") so that it has `Metallic` value of `0.5`. The center object should now be shiny.

**TASK & PLOTS** Generate at least 4 images using Blender for different light positions and include plots of those images with the new material property.

**PLOTS** Use photometric stereo to compute and plot the new (1) albedos, (2) normals, and (3) depth for your modified blender scene.

**QUESTION** Explain (2-3 sentences) why the albedo and normals change. Is there a linear solution to finding the correct normals?

**QUESTION** Find a research paper (I recommend using Google Scholar) that attempts "non-lambertian Photometric Stereo": trying to recover material properties for a non-lamberitan object, like the one shown above. Include a citation of the paper and (in 3–5 sentences) explain how it works and how the approach differs from the lambertian photometric stereo we studied in class.

## P5.3 Structure from Motion

The culminating assignment for the second unit of the course is scene reconstruction, also known as *Structure from Motion* (SfM). You will be asked to implement the simplified version of SfM we discussed in class: Affine Structure from Motion, in which the camera is assumed to have an orthographic projection of the scene.

### P5.3.1 Implementation

The general procedure for Affine SfM was fully derived in class. The general procedure is as follows:

<img src="images/affine_sfm_procedure.png" width="400">

I have provided you with code that completes the first couple of steps: I have detected features for each of the four provided images and computed feature matches using OpenCV. The feature matches, stored in `combined_matches`, are indexed according to (0) feature number, [1] image number, [2] coordinate. So the x-coordinate of detected feature 5 in image 3, would be `combined_matches[5, 3, 0]`.

Second, I have centered the points for each of the images. The resulting centered feature points are stored in the 3-dimensional data structure `matches_norm`. You should use this for constructing the measurement matrix `D`.

**TASK** Implement the procedure for Affine Structure from Motion so that you can compute the motion matrix `M` and the shape matrix `S` as defined in the procedure above. See the lecture slides for how to construct the measurement matrix `D`. 

> Note also that solving for `M` and `S` requires using Singular Value Decomposition; you should verify that you are selecting the correct parts of the decomposed matrices as your output. Refer to the numpy documentation to confirm that you are using it correctly. Alternatively, you can refer to your earlier assignments and confirm that you can get SVD to work properly there. Overall, this question does not require very much code, but you have to set up the matrix `D` correctly and extract the correct parts of that matrix to get `M` and `S`, which can take some thought. One helpful tip is to pay attention to the size of the matrices. I do not recommend trial and error for this. 

The final step of the algorithm above says to *Eliminate affine ambiguity*; I have provided you with code that does this in `correct_affine_ambiguity`, which takes in the `M` and `S` matrices you are to compute and returns the matrices `A` (containing the affine projection matrices) and `X`.

**TASK** Using your newly-computed affine projected matrices (stored in the matrix `A`), you can reproject the 3D points `X` into the image plane. Using the slides from class as a reference, define the function `reproject_image_points(A, X)`, which returns a single matrix `reprojected_image_points` (and has dimensions matching `combined_matches`, with coordinates [0] feature number, [1] image number, [2] coordinate. *Don't forget to re-add the `centers` vector after reprojecting.*

**PLOTS** If your functions are all defined correctly, run the code below under `Plotting: Affine SfM`. It should generate a set of 4 plots with the reprojected points overlaid on top of them; include the set of 4 images in your writeup. The four plots should look similar to this:

<img src="images/affine_sfm_reprojection.png" width="200">


In [ ]:
# First, let's follow the OpenCV tutorial for F-matrix calculation

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from PIL import Image

img1 = cv.imread('SfM/simple_view_c.png', 0) #queryimage # center image
img2 = cv.imread('SfM/simple_view_1.png', 0) #trainimage # left image
img3 = cv.imread('SfM/simple_view_2.png', 0) #trainimage # upper image
img4 = cv.imread('SfM/simple_view_3.png', 0) #trainimage # right image
sift = cv.SIFT_create()


def load_image(filepath):
    img = Image.open(filepath)
    return (np.asarray(img).astype(np.float)/255)[:, :, :3]

img1_color = load_image('SfM/simple_view_c.png')

def get_point_matches(img1, img2):
    """Returns matches as array: (feature track, image, coord)"""

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=10)
    search_params = dict(checks=100)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    good = []
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
        
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_LMEDS)
    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    
    return np.stack((pts1, pts2), axis=1)


def combine_matches(matches_a, matches_b):
    """Assumes that the 0'th image is the same between them."""
    combined_matches = []
    for ii in range(matches_a.shape[0]):
        ma0 = matches_a[ii, 0]
        # Find the match in b
        mi = np.where((matches_b[:, 0] == ma0).all(axis=1))[0]

        # If a match is found, add to the array
        if mi.size > 0:
            ma = matches_a[ii]
            mb = matches_b[int(mi[0])]
            combined_matches.append(np.concatenate(
                (ma, mb[1:]), axis=0))

    return np.array(combined_matches)


def get_match_colors(image_c, combined_matches):
    colors = []
    nm = combined_matches.shape[0]
    for mi in range(nm):
        m = combined_matches[mi, 0, :]
        colors.append(image_c[m[1]-1:m[1]+2,
                              m[0]-1:m[0]+2].sum(axis=0).sum(axis=0)/9)
    
    return colors


# Solve for the Affine Ambiguity
def correct_affine_ambiguity(M, S):
    Atilde = M
    Xtilde = S
    m = Atilde.shape[0]//2
    Am = np.zeros((3*m, 9))
    bm = np.zeros(3*m)
    for mi in range(m):
        Atl = Atilde[2*mi:2*mi+2]
        for ri in range(3):
            for ci in range(3):
                Am[3*mi+0, 3*ri + ci] = Atl[0, ri] * Atl[0, ci]
                bm[3*mi+0] = 1
                Am[3*mi+1, 3*ri + ci] = Atl[1, ri] * Atl[1, ci]
                bm[3*mi+1] = 1
                Am[3*mi+2, 3*ri + ci] = Atl[0, ri] * Atl[1, ci]
                bm[3*mi+2] = 0

    CCT = np.reshape(np.linalg.lstsq(Am, bm, rcond=None)[0], (3, 3))
        
    # Enforce a Positive Semi-Definite Matrix
    # (usually not necessary)
    C = 0.5*(CCT + CCT.T)
    w, v = np.linalg.eig(C)
    CCT = v @ np.diag(np.abs(w)) @ np.linalg.inv(v)

    C = np.linalg.cholesky(CCT)
    
    A = Atilde @ C
    X = np.linalg.inv(C) @ Xtilde
    return A, X


## Now we compute the feature matches
matches_12 = get_point_matches(img1, img2)
matches_13 = get_point_matches(img1, img3)
matches_14 = get_point_matches(img1, img4)

combined_matches = combine_matches(matches_12, matches_13)
combined_matches = combine_matches(combined_matches, matches_14)
colors = get_match_colors(img1_color, combined_matches)

# Plotting code (for visualization)
plt.figure(figsize=(12, 3), dpi=150)
stacked_images = (img1, img2, img3, img4)
num_images = len(stacked_images)
for ii in range(num_images):
    plt.subplot(1, num_images, ii+1)
    plt.imshow(stacked_images[ii], cmap='gray')
    plt.plot(combined_matches[:, ii, 0],
             combined_matches[:, ii, 1],
             '.')

    
## First we normalize the points
# (subtract off average along "point" dimension.)
centers = np.mean(combined_matches, axis=0)
matches_norm = combined_matches - centers


## TASK: use matches_norm to solve for the 3D structure
# Define M and S from the lecture notes
M = None
S = None

## TASK: 
def reproject_image_points(A, X, centers):  
    raise NotImplementedError()


In [ ]:
## Plotting: Affine SfM

if M is None or S is None:
    raise NotImplementedError()
    
print(f"M (before correction):\n {M}")

# Correct for Affine Ambiguity
A, X = correct_affine_ambiguity(M, S)

print(f"A (after correction):\n {A}")

reprojected_image_points = reproject_image_points(A, X, centers)
# Save to file for 3D plotting:
np.save("points.npy", X)
np.save("colors.npy", colors)

# Plotting Code (reprojected points):
plt.figure(figsize=(8, 8), dpi=150)
stacked_images = (img1, img2, img3, img4)
num_images = len(stacked_images)
for ii in range(num_images):
    plt.subplot(2, 2, ii+1)
    plt.imshow(stacked_images[ii], cmap='gray')
    plt.plot(combined_matches[:, ii, 0],
             combined_matches[:, ii, 1],
             '.')
    plt.plot(reprojected_image_points[:, ii, 0],
             reprojected_image_points[:, ii, 1],
             'ro', markerfacecolor='none')

### P5.3.2 Followup Conceptual Questions

I have provided you with some plotting code below in `Plotting Code 3D`, which you can use to view your computed 3D points in 3D. **I recommend you run this code in a terminal window, so that you can view it interactively.** The code above saves the data to file in `points.npy` and `colors.npy`, which the plotting code below reads in before visualization. Note also that I have given the features colors according to their color in the original image.

**PLOTS** Record 4 different perspectives on your scene using the 3D plotting code. One of those views should show that the points on the surface of the central cube are *coplanar* (i.e. generate a view from above or below the central cube so that you can tell that those points are roughly all on the same plane, as we expect).

**QUESTION** What is the relationship between the Affine matrix you have computed and the 3x4 camera projection matrix? Pick one of the input images above. Using the affine transformation matrix you computed for that image, what is the 3x4 camera projection matrix for this image? Include it in your writeup.

**QUESTION** I have provided you with the function `correct_affine_ambiguity` so that the Affine projective matrices produced by our algorithm satisfy certain properties. What are these properties? You may refer to the code itself or the course notes in your answer. It may help you to print the matrices before and after the correction is applied.

In [ ]:
## Plotting Code 3D: Open in interactive terminal so that you can "look around"

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

X = np.load("points.npy")
colors = np.load("colors.npy")
print(colors.shape)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[0, :], X[1, :], X[2, :], c=colors)
ax.set_xlim3d([-750, 750])
ax.set_ylim3d([-750, 750])
ax.set_zlim3d([-750, 750])
plt.show()